In [ ]:
import copy

import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

from scwidgets import (CodeDemo, ParametersBox, CodeChecker, PyplotOutput, ClearedOutput, AnimationOutput)
from widget_code_input import WidgetCodeInput

# Introduction

This example notebook shows all considered variations using the `CodeDemo` class.

## Preliminary helper function

In [ ]:
# utility function for plotting lattice, not essential for understanding the code demo
def plot_lattice(ax, a1, a2, basis=None, alphas=None, s=20, c='red', 
                 lattice_size = 60, head_length = 0.5, head_width= 0.2, width=0.05):
    if basis is None:
        basis = np.array([[0,0]])
    A = np.array([a1, a2])
    # each atom in the basis gets a different basis alpha value when plotted
    if alphas is None:
        alphas = np.linspace(1, 0.3, len(basis))
    for i in range(len(basis)):
        lattice = (np.mgrid[:lattice_size,:lattice_size].T @ A + basis[i]).reshape(-1, 2)
        lattice -= (np.array([lattice_size//2,lattice_size//2]) @ A).reshape(-1, 2)
        ax.scatter(lattice[:,0], lattice[:,1], color=c, s=s, alpha=alphas[i])
        
    ax.fill([0,a1[0],(a1+a2)[0],a2[0]], [0,a1[1],(a1+a2)[1],a2[1]], color=c, alpha=0.2)
    ax.arrow(0,0, a1[0], a1[1],width=width,
             length_includes_head=True,
             fc=c, ec='black')
    ax.arrow(0,0, a2[0], a2[1],width=width,
             length_includes_head=True,
             fc=c, ec='black')

# 1. ParametersBox + Visualizer (PyplotOutput, ClearedOutput)

We can make a code demo with a box of parameters and one or multiple visualization outputs. The demo logic is always written in a `update_visualizers` function which uses the globally defined widgets. The order of input of the `ParamatersBox` instance and the `update_visualizers` function need to be compatible.

## 1.1 ParametersBox + PrintOutput (automatic update)

In [ ]:
example1p1_parameters_box = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
               a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
               a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
               a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

# process code function
def example1p1_update_visualizers(a11, a12, a21, a22, visualizers):
    print_output = visualizers[0]
    
    a1 = np.array([a11, a12])
    a2 = np.array([a21, a22])
    R = np.array([[0,-1],[1,0]])

    # change to the correct expression
    b1 = 2*np.pi*R@a2/(a1@R@a2)
    b2 = 2*np.pi*R@a1/(a2@R@a1)

    with print_output:
        print("Reciprocal vectors")
        print("  b1:", b1)
        print("  b2:", b2)

example1p1_code_demo = CodeDemo(
            input_parameters_box=example1p1_parameters_box,
            visualizers=[ClearedOutput()],
            update_visualizers=example1p1_update_visualizers)
display(example1p1_code_demo)

## 1.2 ParametersBox + PrintOutput (update by button click)

By default the `update_visualizers` function is executed for each change in the paramaters in the `ParametersBox` instance.  For computational demanding functions we do not want to update on each parameter change, but rather first tune all parameters and then update.

In [ ]:
example1p2_parameters_box = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
               a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
               a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
               a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

# We can reuse the function if the logic stays the same, this can be useful if
# one exercise requires the implementation of different algorithms computing the samer result
# (e.g. comparison of efficiency for sorting a list, comparison of accuracy for different time integrator)
example1p2_code_demo = CodeDemo(
            input_parameters_box=example1p2_parameters_box,
            visualizers=[ClearedOutput()],
            update_visualizers=example1p1_update_visualizers, # <--- reuse of function from example 1.1
            update_on_input_parameter_change=False)
display(example1p2_code_demo)

## 1.3 ParametersBox + [PrintOutput, PyplotOutput]

We can also support multiple visualizer outputs

In [ ]:
example1p3_parameters_box = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
              a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
              a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
              a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

# These two lines have to be executed in the same cell otherwise a figure is plotted
# PyplotOutput surpresses the instantaneous plotting of plt.subplots
example1p3_figure, _ = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
example1p3_pyplot_output = PyplotOutput(example1p3_figure)


def example1p3_process(a11, a12, a21, a22, visualizers):
    print_output = visualizers[0]
    pyplot_output = visualizers[1]
    
    axes = pyplot_output.figure.get_axes()
    
    basis = np.array([[0,0]])
    a1 = np.array([a11, a12])
    a2 = np.array([a21, a22])
    R = np.array([[0,-1],[1,0]])

    # change to the correct expression
    b1 = 2*np.pi*R@a2/(a1@R@a2)
    b2 = 2*np.pi*R@a1/(a2@R@a1)

    with print_output:
        print("Reciprocal vectors")
        print("  b1:", b1)
        print("  b2:", b2)

    plot_lattice(axes[0], a1, a2, basis, s=20, c='red')
    plot_lattice(axes[1], b1/(2*np.pi), b2/(2*np.pi), basis, s=20, c='blue')
        
    axes[0].set_title('real space')
    axes[0].set_xlim(-5,5)
    axes[0].set_ylim(-5,5)
    axes[0].set_xlabel("$x$ / Å")
    axes[0].set_ylabel("$y$ / Å")
    
    axes[1].set_title('reciprocal space')
    axes[1].set_xlim(-5,5)
    axes[1].set_ylim(-5,5)
    axes[1].set_xlabel("$k_x/2\pi$ / Å$^{-1}$")
    axes[1].set_ylabel("$k_y/2\pi$ / Å$^{-1}$")


example1p3_cd = CodeDemo(
            input_parameters_box=example1p3_parameters_box,
            visualizers=[ClearedOutput(), example1p3_pyplot_output],
            update_visualizers=example1p3_process)
display(example1p3_cd)


# 2. WidgetCodeInput

## 2.1 WidgetCodeInput + CodeChecker

In [ ]:
example2p1_code_input = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="a1, a2",
        docstring="""
Return the 2D reciprocal unit cell vectors.

:param a1: unit cell vector a1 
:param a2: unit cell vector a2

:return: reciprocal lattice unit cell vectors
""",
        function_body="""

import numpy as np
from numpy import pi

a1 = np.asarray(a1)
a2 = np.asarray(a2)

R = np.array([[0,-1],[1,0]])

# Wrong solution
b1 = 2*pi*a1 
b2 = 2*pi*a2 

# Try correct solution
#b1 = 2*np.pi*R@a2/(a1@R@a2)
#b2 = 2*np.pi*R@a1/(a2@R@a1)


return b1, b2
"""
)

def compute_reciprocal_lattice_vectors(a1, a2):
    R = np.array([[0,-1],[1,0]])
    b1 = 2*np.pi*R@a2/(a1@R@a2)
    b2 = 2*np.pi*R@a1/(a2@R@a1)
    return b1, b2

reference_code_parameters = {
       input_parameters: compute_reciprocal_lattice_vectors(
           np.asarray(input_parameters[0]), np.asarray(input_parameters[1]) 
       )
       for input_parameters in [((0,1), (1,0)), ((1,1), (1,-1)), ((0,2), (2,1))]
}

reciprocal_lattice_code_checker = CodeChecker(reference_code_parameters, equality_function=np.allclose)

example2p1_code_demo = CodeDemo(
            code_input=example2p1_code_input,
            code_checker=reciprocal_lattice_code_checker,
            update_on_input_parameter_change=False
)
display(example2p1_code_demo)

## 2.2 WidgetCodeInput + PyplotOutput

In [ ]:
example2p2_code_input = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="",
        docstring="""
Return the 2D lattice basis

:return: lattice basis
""",
        function_body="""
import numpy as np

basis = np.array([[0, 0], [0.5, 0.5]])

return basis
"""
)


example2p2_parameters_box = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
              a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
              a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
              a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

example2p2_figure, _ = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
example2p2_pyplot_output = PyplotOutput(example2p2_figure)

def example2p2_process(code_input, visualizers):
    pyplot_output = visualizers[0]
    axes = pyplot_output.figure.get_axes()
    
    a1 = np.array([1, 0])
    a2 = np.array([0, 1])

    basis = code_input.get_function_object()()
    b1, b2 = compute_reciprocal_lattice_vectors(a1, a2)
    

    plot_lattice(axes[0], a1, a2, basis, s=20, c='red')
    plot_lattice(axes[1], b1/(2*np.pi), b2/(2*np.pi), basis, s=20, c='blue')
        
    axes[0].set_title('real space')
    axes[0].set_xlim(-5,5)
    axes[0].set_ylim(-5,5)
    axes[0].set_xlabel("$x$ / Å")
    axes[0].set_ylabel("$y$ / Å")
    
    axes[1].set_title('reciprocal space')
    axes[1].set_xlim(-5,5)
    axes[1].set_ylim(-5,5)
    axes[1].set_xlabel("$k_x/2\pi$ / Å$^{-1}$")
    axes[1].set_ylabel("$k_y/2\pi$ / Å$^{-1}$")


example2p2_code_demo = CodeDemo(
            code_input=example2p2_code_input,
            visualizers=[example2p2_pyplot_output],
            update_visualizers=example2p2_process,
            update_on_input_parameter_change=False
)

display(example2p2_code_demo)

## 2.3 WidgetCodeInput + ParametersBox + PyplotOutput (automatic update)

In [ ]:
example2p3_code_input = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="a1, a2",
        docstring="""
Return the 2D reciprocal unit cell vectors.

:param a1: unit cell vector a1 
:param a2: unit cell vector a2

:return: reciprocal lattice unit cell vectors
""",
        function_body="""

import numpy as np
from numpy import pi

a1 = np.asarray(a1)
a2 = np.asarray(a2)

R = np.array([[0,-1],[1,0]])

# Wrong solution
b1 = 2*pi*a1 
b2 = 2*pi*a2 

# Try correct solution
#b1 = 2*np.pi*R@a2/(a1@R@a2)
#b2 = 2*np.pi*R@a1/(a2@R@a1)


return b1, b2
"""
)


example2p3_parameters_box = ParametersBox(
              #fs = Slider(),
              a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
              a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
              a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
              a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

example2p3_figure, _ = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
example2p3_pyplot_output = PyplotOutput(example2p3_figure)

#def example2p3_process(parameters, code_input=None, visualizers=None):
#    parameters['a11']
#    pyplot_output = visualizers[0]


def example2p3_process(a11, a12, a21, a22, code_input=None, visualizers=None):
    pyplot_output = visualizers[0]
    axes = pyplot_output.figure.get_axes()
    
    basis = np.array([[0,0]])
    a1 = np.array([a11, a12])
    a2 = np.array([a21, a22])

    b1, b2 = code_input.get_function_object()(a1, a2)

    plot_lattice(axes[0], a1, a2, basis, s=20, c='red')
    plot_lattice(axes[1], b1/(2*np.pi), b2/(2*np.pi), basis, s=20, c='blue')
        
    axes[0].set_title('real space')
    axes[0].set_xlim(-5,5)
    axes[0].set_ylim(-5,5)
    axes[0].set_xlabel("$x$ / Å")
    axes[0].set_ylabel("$y$ / Å")
    
    axes[1].set_title('reciprocal space')
    axes[1].set_xlim(-5,5)
    axes[1].set_ylim(-5,5)
    axes[1].set_xlabel("$k_x/2\pi$ / Å$^{-1}$")
    axes[1].set_ylabel("$k_y/2\pi$ / Å$^{-1}$")


example2p3_code_demo = CodeDemo(
            code_input=example2p3_code_input,
            input_parameters_box=example2p3_parameters_box,
            visualizers=[example2p3_pyplot_output],
            update_visualizers=example2p3_process
)

display(example2p3_code_demo)


## 2.4 WidgetCodeInput + ParametersBox + PyplotOutput (update on button click)

In [ ]:
example2p4_code_input = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="a1, a2",
        docstring="""
Return the 2D reciprocal unit cell vectors.

:param a1: unit cell vector a1 
:param a2: unit cell vector a2

:return: reciprocal lattice unit cell vectors
""",
        function_body="""

import numpy as np
from numpy import pi

a1 = np.asarray(a1)
a2 = np.asarray(a2)

R = np.array([[0,-1],[1,0]])

# Wrong solution
b1 = 2*pi*a1 
b2 = 2*pi*a2 

# Try correct solution
#b1 = 2*np.pi*R@a2/(a1@R@a2)
#b2 = 2*np.pi*R@a1/(a2@R@a1)

return b1, b2
"""
)


example2p4_parameters_box = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
              a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
              a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
              a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

example2p4_figure, _ = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
example2p4_pyplot_output = PyplotOutput(example2p4_figure)


example2p4_code_demo = CodeDemo(
            code_input=example2p4_code_input,
            input_parameters_box=example2p4_parameters_box,
            visualizers=[example2p4_pyplot_output],
            update_visualizers=example2p3_process,
            update_on_input_parameter_change=False
)

display(example2p4_code_demo)


## 2.5 WidgetCodeInput + CodeChecker + ParametersBox + PyplotOutput (merged check and update button)

In [ ]:
example2p4_code_input = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="a1, a2",
        docstring="""
Return the 2D reciprocal unit cell vectors.

:param a1: unit cell vector a1 
:param a2: unit cell vector a2

:return: reciprocal lattice unit cell vectors
""",
        function_body="""

import numpy as np
from numpy import pi

a1 = np.asarray(a1)
a2 = np.asarray(a2)

R = np.array([[0,-1],[1,0]])

# Wrong solution
b1 = 2*pi*a1 
b2 = 2*pi*a2 

# Try correct solution
#b1 = 2*np.pi*R@a2/(a1@R@a2)
#b2 = 2*np.pi*R@a1/(a2@R@a1)

return b1, b2
"""
)


example2p4_parameters_box = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
              a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
              a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
              a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

example2p4_figure, _ = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
example2p4_pyplot_output = PyplotOutput(example2p4_figure)


example2p4_code_demo = CodeDemo(
            code_input=example2p4_code_input,
            input_parameters_box=example2p4_parameters_box,
            visualizers=[example2p4_pyplot_output],
            update_visualizers=example2p3_process,
            code_checker=reciprocal_lattice_code_checker,
)

display(example2p4_code_demo)


## 2.6 WidgetCodeInput + CodeChecker + ParametersBox + PyplotOutput (separate check and update button, update on button click)

In [ ]:
example2p6_code_input = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="a1, a2",
        docstring="""
Return the 2D reciprocal unit cell vectors.

:param a1: unit cell vector a1 
:param a2: unit cell vector a2

:return: reciprocal lattice unit cell vectors
""",
        function_body="""

import numpy as np
from numpy import pi

a1 = np.asarray(a1)
a2 = np.asarray(a2)

R = np.array([[0,-1],[1,0]])

# Wrong solution
b1 = 2*pi*a1 
b2 = 2*pi*a2 

# Try correct solution
#b1 = 2*np.pi*R@a2/(a1@R@a2)
#b2 = 2*np.pi*R@a1/(a2@R@a1)

return b1, b2
"""
)


example2p6_parameters_box = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
              a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
              a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
              a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'))

example2p6_figure, _ = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
example2p6_pyplot_output = PyplotOutput(example2p6_figure)


example2p6_code_demo = CodeDemo(
            code_input=example2p6_code_input,
            input_parameters_box=example2p6_parameters_box,
            visualizers=[example2p6_pyplot_output],
            update_visualizers=example2p3_process,
            code_checker=reciprocal_lattice_code_checker,
            separate_check_and_update_buttons=True,
            update_on_input_parameter_change=False
)

display(example2p6_code_demo)


# 3. Animation

In [ ]:
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
animation_output = AnimationOutput(fig)

animation_parameters_box = ParametersBox(frequency = (np.pi, 1, 10, 1, r'frequency'))

def animation_update_visualizers(frequency, visualizers):
    ao = visualizers[0]
    ax = ao.figure.get_axes()[0]

    def init():
        line = ax.plot([], [])
        #return (line,)
    
    def animate(i):
        x = np.linspace(0, 2, 1000)
        y = np.sin(2 * np.pi/frequency * (x - 0.01 * i))
        ax.clear()
        ax.plot(x, y, lw=2)
        ax.set_xlim((0, 2))
        ax.set_ylim((-2, 2))
        
    anim = FuncAnimation(fig, animate,
                                   frames=100, interval=20,
                                   blit=True)
    ao.animation = anim
    
    

animation_code_demo = CodeDemo(
    code_input=None,
    input_parameters_box=animation_parameters_box,
    visualizers=[animation_output],
    update_visualizers=animation_update_visualizers,
    update_on_input_parameter_change=False
)

display(animation_code_demo)

# 4. Chemiscope

## 4.1 Update chemiscope visualizer parameters

In [ ]:
import chemiscope
import ase.lattice
fcc_al = ase.lattice.cubic.FaceCenteredCubic('Al')

cutoff_parameters_box = ParametersBox(cutoff = (3., 0.1, 10., 0.1, r'$r_c / Å$'))
fcc_al_widget = chemiscope.show(frames = [fcc_al], mode="structure", 
                        environments=chemiscope.all_atomic_environments(fcc_al),
                        settings={"structure":[{"unitCell":True,"supercell":{"0":3,"1":3,"2":3}}]}
                       )

# process code function
def chemiscope_update_visualizers(cutoff, visualizers=None):
    chemiscope_widget = visualizers[0]
    
    chemiscope_widget.settings={"structure": [{"environments": {"cutoff":  cutoff}}]}


chemiscope_code_demo = CodeDemo(
            input_parameters_box=cutoff_parameters_box,
            visualizers=[fcc_al_widget],
            update_visualizers=chemiscope_update_visualizers)
display(chemiscope_code_demo)

## 4.2 Update chemiscope visualizer atomic structure

In [ ]:
example4p2_code_input = WidgetCodeInput(
        function_name="return_structure", 
        function_parameters="",
        docstring="""
Return structure of interest to visualize

:return: atomic structure visualizable by chemiscope (ase.Atoms)
""",
        function_body="""
import ase.lattice
fcc_al = ase.lattice.cubic.FaceCenteredCubic('Al')
return fcc_al
"""
)

def chemiscope_update_visualizers_structure(code_input, visualizers):
    cleared_output = visualizers[0]
    frame = code_input.get_function_object()()
    with cleared_output:
        chemiscope_widget = chemiscope.show(frames = [frame], mode="structure")
        display(chemiscope_widget)


    

example4p2_code_demo = CodeDemo(
            code_input=example4p2_code_input,
            input_parameters_box=None,
            visualizers=[ClearedOutput()],
            update_visualizers=chemiscope_update_visualizers_structure,
            code_checker=None,
            update_on_input_parameter_change=False
)

display(example4p2_code_demo)